### Non-Deterministic Primality Test

- What are deteriministic vs non-deterministic algorithms?
    - Deterministic: Tests primality with 100% accuracy
    - Non-deterministic: Tests primality with some small amount of error (e.g. Fermat's primality test)
    - So far, we've only looked at deterministic test

- Why study primality test with errors?
    - Because deterministic tests run in $O(\sqrt{N})$ time, which is extremely slow
    - Imagine you are asked to test primality of numbers up to $10^{19}$. WIth $O(\sqrt{N})$, you need $10^{9+}$ operations!     
    - Non-deterministic tests run in $O(\log{N})$ time

### Problem

- Problem: You are given 2 integers $N$ and $K$. Can $N$ be represented as a sum of $K$ prime numbers?
    - $1 \le N, K \le 10^{12}$

#### 1. Fermat Primality Test

- Based on Fermat's little theorem (FLT) (we also used this in Euler's Totient Function)
    - FLT: For a prime number $p$ and a coprime integer $a$, it must be true that $a^{p-1} \equiv 1 \mod p$
    - $p$ does not hold for composite numbers **in general**
        - Can we use this to conclude whether a number if prime? i.e. if FLT holds, then number is prime

- Approach
    - Choose $a$ such that $2 \le a \le p-2$ and compute $a^{p-1}$
        - Why do we choose $a$ only up to $a=p-2$?
            - $a^n \mod p = (a \mod p)^n \mod p$, by modular arithmetic
            - If $a = p-1$, $p-1 \mod p = p-1$
            - Then $a^{n} \mod p = (a \mod p)^n \mod p = ((p-1) \mod p)^n \mod p = (p-1)^n \mod p = (p-1)^{p-1} \mod p$
            - We know that $p$ must be odd, because every prime is odd. Thereby, $p-1$ must be even
            - Notice that $(p-1)^{p-1}$ is just binomial expansion $S$. 
                - Every term of the expansion must contain some polynomial of $p$ except the last term, which will be +1 since $p-1$ is even
            - Also keep in mind that $a+b \mod p = (a \mod p + b \mod p) \mod p$
            - So the binomial expansion $S \mod p$ must be 1
            - As such, using $a=p-1$ when $p$ is odd will always give us $a^{p-1} \equiv 1 \mod p$, which is useless as a primality test

    - Then we compute $r = a^{p-1} \mod p$
    - If the $r \neq 1$, we can conclude with certainty that $p$ is not a prime!! So $a$ is known as Fermat's witness for compositeness of $p$
    - If $r = 1$, we **cannot** conclude with certainty that $p$ is prime! Because there are composite numbers $p'$ that have this property where $a^{p' - 1} \equiv 1 \mod p' $  
        - So now $p$ is known as a probable prime
        - However, it is extremely unlikely for multiple randomly chosen $a$ to meet this property if $p'$ is composite. So if we perform multiple iterations of this, we can become extremely certain about the primality of $p$

In [55]:
import random

def probable_prime_fermat(n: int, iter: int = 10) -> bool:
    '''
    NOTE: BINARY EXPONENTIATION is needed when taking power, so don't blindly take (a ** n-1) because you will get an overflow
    '''
    if n < 4:
        return ((n == 2) or (n == 3))
    
    for i in range(iter):
        a = random.randint(2, n-1)
        # a = n-1
        if pow(a, n-1,n) != 1:
            return False
    return True

probable_prime_fermat(int(9))

True

- For numbers $p$ that are not prime, yet Fermat's little theorem holds for some value $a$, then $a$ is known as a Fermat Liar, and $p$ is a Fermat PseudoPrime

- **PROBLEM WITH THIS APPROACH: Carmichael Numbers**
    - There are some special non-prime numbers (let's call them $C$) where the condition FLT holds $a^{C-1} \equiv 1 \mod C$ holds true for all $\gcd(a, C) \neq 1$
    - These are called Carmichael numbers
    - In other words, if you use Fermat's primality test to test for the primality of a Carmichael number, you can end up assuming the number is prime when it isn't
    - Thankfully, there are only 646 such numbers up to $10^9$


#### 2. Miller-Rabin Primality Test

- Based on Fermat's Little Theorem 

- How does Miller-Rabin Test work?
    - For any number $N$, we can write $N-1 = 2^s \cdot d$, where $s$ is an integer, and $d$ is an odd integer
        - e.g. $7 = 2^0 \cdot 7$
    - By Fermat's Little Theorem, $a^{p-1} \equiv 1 \mod p$
    - Combining both facts
        $$\begin{aligned}
            a^{p-1} \equiv 1 \mod p &\Leftrightarrow a^{2^s \cdot d} \equiv 1 \mod p \\
            &\Leftrightarrow a^{2^s \cdot d} - 1 \equiv 0 \mod p \\
            &\Leftrightarrow (a^{2^{s-1} \cdot d} + 1)(a^{2^{s-1} \cdot d} - 1) \equiv 0 \mod p & a^{2^s \cdot d} - 1 \text{ is } a^2 - b^2\\
            &\Leftrightarrow (a^{2^{s-1} \cdot d} + 1)(a^{2^{s-2} \cdot d} + 1)(a^{2^{s-2} \cdot d} - 1) \equiv 0 \mod p  \\
            &\Leftrightarrow (a^{2^{s-1} \cdot d} + 1)(a^{2^{s-2} \cdot d} + 1)...(a^{d} + 1)(a^{d} - 1) \equiv 0 \mod p  \\
            \end{aligned}$$
    - We have transformed Fermat's little theorem into a polynomial product
        - So if the number $p$ is truly prime, it must divide one of the factors $(a^{2^{k} \cdot d} + 1)$ for some value of $k$
        - So if $p$ does not divide any of these numbers, it cannot be prime
        - But if $p$ divides one or more of these numbers, it is only a **probable prime**

    - Miller-Rabin test is a stricter test than Fermat's primality test
        - Intuitively, Miller Rabin test does not have a corresponding set of Carmichael numbers, so is less susceptible to that pitfall

In [86]:
def check_composite(n: int, a: int, d: int, s: int):
    x: int = pow(a, d, n)
    
    ## Check the last 2 terms in the product; a^d+1 and a^d - 1. 
    ## If a^d mod n returns n-1. then a^d+1 must return 0
    ## If a^d mod n returns 1. then a^d-1 must return 0
    ## Then we know for sure that the number is prime, by miller rabin, since n divides one of the products 
    if ((x == 1) or (x == (n-1))): 
        return False
    
    ## After checking the last 2 terms, check the rest of the polynomials in a loop
    for _ in range(s):
        x = pow(x, 2, n) ##At the first loop, x=a^d. By squaring it, we get a^2d. Then a^4d and so on

        ## Since all other polynomial terms are a^{...} + 1, if they are divisible by p, then a^{...} must be 1 less than a number divisible by p
        if x == (n-1):
            ## Not a prime
            return False 
    return True


def miller_rabin(n: int, iter: int = 5) -> bool:
    '''
    We want to test if input `n` is prime
    '''
    if (n < 4):
        return ((n == 2) or (n == 3))

    ## Express n-1 as 2^{s} . d
    s = 0
    d = n-1

    ### bitwise & that returns true if d is odd
    ### After this loop, it is guaranteed that 2^{s} . d = n-1
    while not (d & 1):
        s += 1
        d >>= 1 #bitwise operation, rightshift d by 1, equivalent to floor division of d by 2
    
    for _ in range(iter):
        a = random.randrange(2, n-1) #as proved in the last section, you cannot have a = n-1, so a can only go from 2 to n-2
        if check_composite(n, a, d, s):
            return False

    return True
    
miller_rabin(571)

True

#### 3. Deterministic Miller-Rabin Primality Test 

- If choosing 32 bit number, you can get a deterministic Miller-Rabin test if you use 2,3,5,7 as the base (i.e. as $a$)
- If choosing 64 bit number, you can get a deterministic Miller-Rabin test if you use 2,3,5,7,...37 as the base (i.e. as $a$)
- Because the earliest number that gives the wrong answer is bigger than 32 bits. It was proved by brute force